In [1]:
import pandas as pd
import re
import torch
import numpy as np
import torch
import transformers
from ast import literal_eval
import json
import pandas as pd
import random


pd.set_option('display.max_columns', None)

In [6]:
df1 = pd.read_csv("add1.csv")
df2 = pd.read_csv("add2.csv")

In [7]:
df1

,id,paragraph,question,choices,answer,category,question_plus
0,augmented-0,“19세기말 일본은 폐쇄적인 에도 시대를 지나 메이지 유신을 통해 급속한 근대화를 ...,위 글에서 설명된 일본의 근대화 과정에서 나타난 주요 사회적 문제는 무엇으로 분석될...,"['민주주의 제도의 확립', '농민과 노동자의 경제적 불만', '전통 문화의 부흥'...",2,사회-문화,NaN
1,augmented-1,교육부는 최근 한국의 고등학교 교육과정 개편을 통해 '진로 탐색' 과목을 신설하겠다...,교육부가 신설 예정인 '진로 탐색' 과목의 목적은 무엇인가?,"['학생들이 다양한 직업 세계를 이해하기 위해서', '취업률을 높이기 위해서', '...",1,사회-교육개혁,NaN
2,augmented-2,"“[한국사] 조선시대 중기에는 사대부와 농민 사이의 갈등이 심화되었고, 이는 다양한...",위 글에서 언급된 '대동법'의 시행 배경은 무엇과 관련이 있습니까?,"['양반의 세금 부담 증가', '농민의 경제적 안정 추구', '사대부의 권력 강화 ...",2,사회-한국사,NaN
3,augmented-3,"최근 한국의 농산물 수출이 증가하는 가운데, 특히 고품질 쌀과 김치의 수출이 두드러...",최근 한국의 농산물 수출이 증가한 주된 이유는 무엇인가?,"['고품질 쌀의 수출 증가', '유럽연합과의 무역협정 체결', '김치 수출의 감소'...",1,사회-경제,NaN
4,augmented-4,"20세기 초, 한국은 일본의 식민지 지배를 받으며 수많은 저항 운동이 일어났다. 특...",3.1 운동이 일어난 해는 언제인가?,"['1918년', '1919년', '1920년', '1930년', '1945년']",2,사회-한국사,NaN
...,...,...,...,...,...,...,...
358,augmented-493,"작가의 소설 속 주인공은 고독 속에서 자신의 정체성을 탐구하면서, 작은 물건에 대해...",이 지문에서 주인공의 행동은 주로 무엇을 나타내고 있는가?,"['상징적 존재', '존재론적 갈등', '내적 성장', '사회적 격리', '정체성 ...",3,국어-문학,NaN
359,augmented-494,“다양한 감정을 표현하는 방법에는 여러 가지가 있습니다. 특히 <감정의 미학>의 저...,김혜수 작가가 최근에 저술한 책의 제목은 무엇인가?,"['감정의 미학', '내 마음을 그리는 여정', '작은 사건들의 힘', '일상의 감...",2,국어-문학,NaN
360,augmented-496,최근 한국의 한 출판사는 독서 촉진을 위한 새로운 프로그램인 ‘읽기와 나누기’를 개...,'읽기와 나누기' 프로그램의 주된 목표는 무엇인가?,"['참가자들에게 독서법을 교육하는 것', '독서의 즐거움을 나누고 협력적인 독서 문...",2,국어-독서,NaN
361,augmented-497,"최근 미디어의 발달과 함께 고전 문학의 현대적 해석이 주목받고 있다. 특히, 다양한...","최근 다양한 현대 예술 작품에서 재조명되고 있는 고전 문학 작가 중, 제인 오스틴의...","['변신', '오만과 편견', '햄릿', '작은 아씨들', '대지의 아들']",2,국어-문학,NaN


In [8]:
df2

,id,paragraph,question,choices,answer,category,question_plus
0,augmented-0,"""1876년, 조선은 일본과 강화도조약을 체결하게 되며, 이로 인해 조선은 일본과의...",위 지문에 설명된 사건의 결과로 나타났던 조선 사회의 특징은 무엇인가?,"['국내 산업의 발전', '민족 자각의 증대', '외교 관계의 다변화', '경제적 ...",2,사회-역사,NaN
1,augmented-1,최근 정부는 청소년의 사회 참여를 촉진하기 위해 공공기관에서의 인턴십 기회를 확대하...,정부의 청소년 인턴십 정책에 대한 전문가의 긍정적인 견해는 무엇인가?,"['직업 경험을 쌓을 수 있어 진로 설계에 도움이 된다.', '모든 청소년에게 강제...",1,사회-정치와법,NaN
2,augmented-2,"“19세기 중반, 서양 제국주의가 아시아와 아프리카에 확장되면서 원주율의 불평등한 ...",위 글에서 언급된 19세기 중반의 제국주의 확장은 어떤 경제적 구조와의 관련성을 강...,"['산업화된 경제 체제', '자유 시장 경제 원칙', '사회주의 경제 모델', '농...",1,사회-세계사,NaN
3,augmented-3,"최근 연구에 따르면, 한국의 청년층이 가장 많이 소비하는 여가 활동은 온라인 게임과...",최근 한국 청년층의 여가 소비 패턴 변화에서 가장 두드러진 현상은 무엇인가?,"['온라인 게임과 영화 감상 증가', '전통적인 여가 활동 감소', '독서 비율 증...",1,사회-문화,NaN
4,augmented-4,"1960년대 후반, 한국은 산업화와 도시화가 급속히 진행되면서 농촌 인구의 대도시로...",한국에서 산업화와 도시화가 급속히 진행하기 시작한 시기는 언제인가?,"['1950년대', '1960년대', '1970년대', '1980년대', '1990...",2,사회-사회 구조와 변화,NaN
...,...,...,...,...,...,...,...
178,augmented-214,"""하늘 위에는 수많은 별들이 반짝이며 그들 각각이 특별한 이야기를 지니고 있다. 고...",다음 중 위 지문에서 설명하고 있는 별자리의 역할과 관련된 내용으로 적절하지 않은 것은?,"['농업과 계절의 변화를 예측하는 데 사용되었다', '항해에 있어 방향을 제시하였다...",4,국어-문학,NaN
179,augmented-215,고등학교 졸업을 앞둔 은주가 대학입학 전형을 준비하면서 심리학과와 문학과 중 어떤 ...,은주가 두 전공을 고민하게 된 주된 이유는 무엇인가요?,"['대학 진학에 대한 두려움', '자신의 진로에 대한 관심', '부모님의 권유', ...",2,국어-독서,NaN
180,augmented-217,"바람이 불어오는 방향을 알지 못하는 작은 나뭇잎은, 자유를 찾아 떠나는 여행자와 비...",이 지문에서 가장 강조하고 있는 주제는 무엇인가?,"['자연의 힘에 대한 경외', '자기 발견의 과정', '여행의 중요성', '자유의 ...",2,국어-독서,NaN
181,augmented-218,한국의 전통 음식 중 하나인 비빔밥은 그 고유의 색상과 영양가로 유명하다. 이 음식...,비빔밥의 전통적인 형태 중 하나로 올바른 것은 무엇인가?,"['매운탕', '비빔국수', '돌솥비빔밥', '김치찌개', '진주비빔밥']",3,국어-문화,NaN


In [9]:
import pandas as pd

# 데이터프레임 합치기
df_combined = pd.concat([df1, df2], ignore_index=True)


In [10]:
df_combined

,id,paragraph,question,choices,answer,category,question_plus
0,augmented-0,“19세기말 일본은 폐쇄적인 에도 시대를 지나 메이지 유신을 통해 급속한 근대화를 ...,위 글에서 설명된 일본의 근대화 과정에서 나타난 주요 사회적 문제는 무엇으로 분석될...,"['민주주의 제도의 확립', '농민과 노동자의 경제적 불만', '전통 문화의 부흥'...",2,사회-문화,NaN
1,augmented-1,교육부는 최근 한국의 고등학교 교육과정 개편을 통해 '진로 탐색' 과목을 신설하겠다...,교육부가 신설 예정인 '진로 탐색' 과목의 목적은 무엇인가?,"['학생들이 다양한 직업 세계를 이해하기 위해서', '취업률을 높이기 위해서', '...",1,사회-교육개혁,NaN
2,augmented-2,"“[한국사] 조선시대 중기에는 사대부와 농민 사이의 갈등이 심화되었고, 이는 다양한...",위 글에서 언급된 '대동법'의 시행 배경은 무엇과 관련이 있습니까?,"['양반의 세금 부담 증가', '농민의 경제적 안정 추구', '사대부의 권력 강화 ...",2,사회-한국사,NaN
3,augmented-3,"최근 한국의 농산물 수출이 증가하는 가운데, 특히 고품질 쌀과 김치의 수출이 두드러...",최근 한국의 농산물 수출이 증가한 주된 이유는 무엇인가?,"['고품질 쌀의 수출 증가', '유럽연합과의 무역협정 체결', '김치 수출의 감소'...",1,사회-경제,NaN
4,augmented-4,"20세기 초, 한국은 일본의 식민지 지배를 받으며 수많은 저항 운동이 일어났다. 특...",3.1 운동이 일어난 해는 언제인가?,"['1918년', '1919년', '1920년', '1930년', '1945년']",2,사회-한국사,NaN
...,...,...,...,...,...,...,...
541,augmented-214,"""하늘 위에는 수많은 별들이 반짝이며 그들 각각이 특별한 이야기를 지니고 있다. 고...",다음 중 위 지문에서 설명하고 있는 별자리의 역할과 관련된 내용으로 적절하지 않은 것은?,"['농업과 계절의 변화를 예측하는 데 사용되었다', '항해에 있어 방향을 제시하였다...",4,국어-문학,NaN
542,augmented-215,고등학교 졸업을 앞둔 은주가 대학입학 전형을 준비하면서 심리학과와 문학과 중 어떤 ...,은주가 두 전공을 고민하게 된 주된 이유는 무엇인가요?,"['대학 진학에 대한 두려움', '자신의 진로에 대한 관심', '부모님의 권유', ...",2,국어-독서,NaN
543,augmented-217,"바람이 불어오는 방향을 알지 못하는 작은 나뭇잎은, 자유를 찾아 떠나는 여행자와 비...",이 지문에서 가장 강조하고 있는 주제는 무엇인가?,"['자연의 힘에 대한 경외', '자기 발견의 과정', '여행의 중요성', '자유의 ...",2,국어-독서,NaN
544,augmented-218,한국의 전통 음식 중 하나인 비빔밥은 그 고유의 색상과 영양가로 유명하다. 이 음식...,비빔밥의 전통적인 형태 중 하나로 올바른 것은 무엇인가?,"['매운탕', '비빔국수', '돌솥비빔밥', '김치찌개', '진주비빔밥']",3,국어-문화,NaN


In [12]:
# DataFrame에서 컬럼 삭제
df_combined = df_combined.drop(columns=['category'])

In [13]:
nan_counts = df_combined.isnull().sum()
nan_counts

id                 0
paragraph          0
question           0
choices            0
answer             0
question_plus    546
dtype: int64

In [14]:
df_combined

,id,paragraph,question,choices,answer,question_plus
0,augmented-0,“19세기말 일본은 폐쇄적인 에도 시대를 지나 메이지 유신을 통해 급속한 근대화를 ...,위 글에서 설명된 일본의 근대화 과정에서 나타난 주요 사회적 문제는 무엇으로 분석될...,"['민주주의 제도의 확립', '농민과 노동자의 경제적 불만', '전통 문화의 부흥'...",2,NaN
1,augmented-1,교육부는 최근 한국의 고등학교 교육과정 개편을 통해 '진로 탐색' 과목을 신설하겠다...,교육부가 신설 예정인 '진로 탐색' 과목의 목적은 무엇인가?,"['학생들이 다양한 직업 세계를 이해하기 위해서', '취업률을 높이기 위해서', '...",1,NaN
2,augmented-2,"“[한국사] 조선시대 중기에는 사대부와 농민 사이의 갈등이 심화되었고, 이는 다양한...",위 글에서 언급된 '대동법'의 시행 배경은 무엇과 관련이 있습니까?,"['양반의 세금 부담 증가', '농민의 경제적 안정 추구', '사대부의 권력 강화 ...",2,NaN
3,augmented-3,"최근 한국의 농산물 수출이 증가하는 가운데, 특히 고품질 쌀과 김치의 수출이 두드러...",최근 한국의 농산물 수출이 증가한 주된 이유는 무엇인가?,"['고품질 쌀의 수출 증가', '유럽연합과의 무역협정 체결', '김치 수출의 감소'...",1,NaN
4,augmented-4,"20세기 초, 한국은 일본의 식민지 지배를 받으며 수많은 저항 운동이 일어났다. 특...",3.1 운동이 일어난 해는 언제인가?,"['1918년', '1919년', '1920년', '1930년', '1945년']",2,NaN
...,...,...,...,...,...,...
541,augmented-214,"""하늘 위에는 수많은 별들이 반짝이며 그들 각각이 특별한 이야기를 지니고 있다. 고...",다음 중 위 지문에서 설명하고 있는 별자리의 역할과 관련된 내용으로 적절하지 않은 것은?,"['농업과 계절의 변화를 예측하는 데 사용되었다', '항해에 있어 방향을 제시하였다...",4,NaN
542,augmented-215,고등학교 졸업을 앞둔 은주가 대학입학 전형을 준비하면서 심리학과와 문학과 중 어떤 ...,은주가 두 전공을 고민하게 된 주된 이유는 무엇인가요?,"['대학 진학에 대한 두려움', '자신의 진로에 대한 관심', '부모님의 권유', ...",2,NaN
543,augmented-217,"바람이 불어오는 방향을 알지 못하는 작은 나뭇잎은, 자유를 찾아 떠나는 여행자와 비...",이 지문에서 가장 강조하고 있는 주제는 무엇인가?,"['자연의 힘에 대한 경외', '자기 발견의 과정', '여행의 중요성', '자유의 ...",2,NaN
544,augmented-218,한국의 전통 음식 중 하나인 비빔밥은 그 고유의 색상과 영양가로 유명하다. 이 음식...,비빔밥의 전통적인 형태 중 하나로 올바른 것은 무엇인가?,"['매운탕', '비빔국수', '돌솥비빔밥', '김치찌개', '진주비빔밥']",3,NaN


In [15]:
df_combined.to_csv("prompt_data.csv", index=False)